In [1]:
import inspect
import json
import time
import numpy as np
from functools import wraps

class Platform:
    def __init__(self, algorithm, processing, table_path):
        self.algorithm = algorithm
        self.processing = processing
        self.table_path = table_path
        self.logs = False

        self.benchmark = {"name" : type(self.algorithm).__name__}

    def start(self, run_type="test", logs=False):
        self.logs = logs

        if run_type == 'test':
            self.test()
        elif run_type == 'compress':
            self.preprocess_and_compress()
        elif run_type == 'decompress':
            self.decompress_and_postprocess()

    def measure_time(flag):
        def decorator(func):
            def wrapper(self, *args, **kwargs):
                start_time = time.time()
                result = func(self, *args, **kwargs)
                end_time = time.time()

                if getattr(self, flag):
                    print(f"Function {func.__name__} took {end_time - start_time} seconds to execute.")
                    self.benchmark[func.__name__ + "_time"] = end_time - start_time

                return result
            return wrapper
        return decorator

    @measure_time("logs")
    def preprocess_and_compress(self):
        preprocessed_table = self.processing.do_preprocess(self.table_path)
        compressed_table = self.algorithm.compress(preprocessed_table)
        if self.logs:
            compression_rate = self.get_compression_rate(compressed_table)
            self.benchmark["compression_rate"] = compression_rate
        return compressed_table

    @measure_time("logs")
    def decompress_and_postprocess(self, compressed_table):
        decompressed_table = self.algorithm.decompress(compressed_table)
        table = self.processing.do_postprocess(decompressed_table)

        if self.logs:
            loss_rate = self.get_loss_rate(table)
            self.benchmark["loss_rate"] = loss_rate

        return table

    @measure_time("logs")
    def test(self):
        compressed_data = self.preprocess_and_compress()
        decompressed_data = self.decompress_and_postprocess(compressed_data)

    def get_compression_rate(self, compressed_table):
        print("I am dummy: ", inspect.currentframe().f_code.co_name)
        return 100

    def get_loss_rate(self, table):
        print("I am dummy: ", inspect.currentframe().f_code.co_name)
        return 0.0



In [12]:
def json_reader(config_path):
    with open(config_path, 'r') as f:
        config = f.load()

    return Platform(config["Algorithm"], )

SyntaxError: incomplete input (103433785.py, line 2)

In [11]:
Algos = [DummyCompressionAlgorithm(),DummyCompressionAlgorithm(),DummyCompressionAlgorithm()]
Processes = [DummyProcessing(), DummyProcessing(), DummyProcessing()]
for algo in Algos:
    for proc in Processes:
        a = Platform(algo, proc, "...")
        a.start("compress", logs=True)
        print("---------------------")

I am dummy compression
I am dummy:  get_compression_rate
Function preprocess_and_compress took 2.0020992755889893 seconds to execute.
---------------------
I am dummy compression
I am dummy:  get_compression_rate
Function preprocess_and_compress took 2.01265025138855 seconds to execute.
---------------------
I am dummy compression
I am dummy:  get_compression_rate
Function preprocess_and_compress took 2.004348039627075 seconds to execute.
---------------------
I am dummy compression
I am dummy:  get_compression_rate
Function preprocess_and_compress took 2.012227773666382 seconds to execute.
---------------------
I am dummy compression
I am dummy:  get_compression_rate
Function preprocess_and_compress took 2.0118424892425537 seconds to execute.
---------------------
I am dummy compression
I am dummy:  get_compression_rate
Function preprocess_and_compress took 2.001253604888916 seconds to execute.
---------------------
I am dummy compression
I am dummy:  get_compression_rate
Function pre

In [ ]:
#1. replace json with arguments of the class | DONE
#2. do not keep table in class instance | DONE
#3. change names of the Platform to more relevant
#4. rename compression to compress | DONE
#5. add ability to not compress tables every time Platform being created | I can just pass already preprocessed table to Platform
#and also pass the flush preprocessing class which do nothing

In [2]:
import abc

class CompressionAlgorithm(abc.ABC):
    def __init__(self):
        ...
    @property
    @abc.abstractmethod
    def name(self):
        ...

    @abc.abstractmethod
    def compress(self, table):
        ...

    @abc.abstractmethod
    def decompress(self, compressed_data):
        ...

class LearningCompressionAlgorithm(CompressionAlgorithm):

    @abc.abstractmethod
    def fit(self, training_data):
        ...

class NonLearningCompressionAlgorithm(CompressionAlgorithm):
    ...

class LosslessCompressionAlgorithm(CompressionAlgorithm):
    ...

class LossyCompressionAlgorithm(CompressionAlgorithm):
    ...

In [3]:
class DummyCompressionAlgorithm(LosslessCompressionAlgorithm):
    name = 'Dummy Compression Algorithm'
    def compress(self, table):
        time.sleep(2)
        print("I am dummy compression")
        return table
    def decompress(self, compressed_data):
        time.sleep(1)
        print("I am dummy decompression")
        return compressed_data

In [4]:
class Processing(abc.ABC):
    def __init__(self):
        ...
    @property
    @abc.abstractmethod
    def name(self):
        ...

    @abc.abstractmethod
    def do_preprocess(self, raw_table):
        ...

    @abc.abstractmethod
    def do_postprocess(self, processed_table):
        ...

In [5]:
class DummyProcessing(Processing):
    name = "Dummy Pre/Post-Processing"
    def do_preprocess(self, raw_table):
        return raw_table
    def do_postprocess(self, processed_table):
        return processed_table
